In [1]:
from config import *

In [2]:
levelups_ref = pd.read_csv("../data/levelups_panel_2.csv", encoding="utf-8")
# levelups_ref = pd.read_csv("../data/levelups_panel_3.csv", encoding="utf-8") # work-in-progress (main quest)
levelups_ref["event_datetime"] = pd.to_datetime(levelups_ref["event_datetime"])
levelups_ref = levelups_ref.sort_values(["player_slug","level_from"]).reset_index(drop=True)

In [3]:
levelups_ref["player_slug"].nunique()

59342

In [4]:
# 일부 레벨업이 관찰되지 않는 플레이어가 샘플에서 제외되었는지 마지막 체크
a = levelups_ref.groupby(["player_slug","character_slug"])["level_from"].max().reset_index()
b = levelups_ref.groupby(["player_slug","character_slug"]).size().to_frame("n").reset_index()
l_error = pd.merge(a,b).query("level_from != n").player_slug.unique()
print(len(l_error))

0


In [5]:
levelups_ref = levelups_ref.query("player_slug not in @l_error").reset_index(drop=True)
levelups_ref["player_slug"].nunique()

59342

* Total hours played

In [5]:
# levelups_ref["playtime_total"] = levelups_ref[["playtime_main","playtime_side"]].sum(axis=1)
# levelups_ref["playtime_total_cum"] = levelups_ref.groupby(["player_slug","character_slug"])["playtime_total"].cumsum() # cum

* Highest character level

In [6]:
tmp = levelups_ref.groupby(["player_slug","character_slug"])["level_to"].max().to_frame("highest_level").reset_index()
levelups_ref = pd.merge(tmp, levelups_ref, how="left")

* Drop out status (현재 플레이 중인 단계인지, 아니면 게임을 그만두었는지 status check)

In [7]:
# if last level up happened in a week, then playing
tmp = np.where(levelups_ref["level_to"]<levelups_ref["highest_level"], "complete",
               np.where((datetime.datetime.strptime(window_eday,"%Y-%m-%d") - levelups_ref["event_datetime"]).apply(lambda x: x.total_seconds()) < (3600*24*7), "playing",
                        "drop"))

In [8]:
pd.Series(tmp).value_counts()

complete    2407655
drop          58759
playing         583
dtype: int64

In [9]:
tmp_ = levelups_ref[["player_slug","character_slug"]].copy()
tmp_["status"] = tmp
levelups_ref = pd.concat([tmp_, levelups_ref.drop(columns=["player_slug","character_slug"])], axis=1)

In [10]:
# switch highest_level and status position
tmp = levelups_ref["status"]
levelups_ref.drop(columns=["status"], inplace=True)
levelups_ref.insert(levelups_ref.columns.tolist().index("highest_level")+1, "status", tmp)

* Total purchases (Response variable)

In [11]:
levelups_ref

,player_slug,character_slug,highest_level,status,level_from,level_to,event_datetime,death,ad_revival,ad_item,ad_total,purchase,event_datetime_prev,interval,interval_cum,playtime_boss,playtime_dungeon,playtime_tower,playtime_pvp,playtime_farm,playtime_survival,playtime_side
0,2021-10-31T16:01:57.631Z,킹덤,7,complete,1,2,2021-11-04 23:42:28,0.0,0.0,0.0,0.0,0.0,2021-11-04 23:41:55,0.009444,0.009444,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-10-31T16:01:57.631Z,킹덤,7,complete,2,3,2021-11-13 07:50:45,0.0,0.0,0.0,0.0,0.0,2021-11-04 23:42:28,200.138333,200.147778,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-10-31T16:01:57.631Z,킹덤,7,complete,3,4,2021-11-13 07:52:19,0.0,0.0,0.0,0.0,0.0,2021-11-13 07:50:45,0.026389,200.174167,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-10-31T16:01:57.631Z,킹덤,7,complete,4,5,2021-11-13 13:11:55,0.0,0.0,0.0,0.0,0.0,2021-11-13 07:52:19,5.326944,205.501111,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-10-31T16:01:57.631Z,킹덤,7,complete,5,6,2021-11-13 13:15:02,0.0,0.0,0.0,0.0,0.0,2021-11-13 13:11:55,0.052222,205.553333,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466992,2022-03-30T14:36:22.522Z,킹키킹,8,complete,3,4,2022-03-30 23:44:15,0.0,0.0,0.0,0.0,0.0,2022-03-30 23:42:14,0.033889,0.084444,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2466993,2022-03-30T14:36:22.522Z,킹키킹,8,complete,4,5,2022-03-30 23:46:19,0.0,0.0,0.0,0.0,0.0,2022-03-30 23:44:15,0.034722,0.119167,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2466994,2022-03-30T14:36:22.522Z,킹키킹,8,complete,5,6,2022-03-30 23:47:29,0.0,0.0,0.0,0.0,0.0,2022-03-30 23:46:19,0.019722,0.138889,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2466995,2022-03-30T14:36:22.522Z,킹키킹,8,complete,6,7,2022-03-30 23:48:48,0.0,0.0,0.0,0.0,0.0,2022-03-30 23:47:29,0.022222,0.161111,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
tmp = levelups_ref.groupby(["player_slug","character_slug"])[["purchase","ad_total"]].sum().reset_index()
tmp.columns = ["player_slug","character_slug","total_purchase","total_ad"]
tmp = pd.merge(levelups_ref[["player_slug","character_slug"]], tmp, how="left")

levelups_ref = pd.concat([tmp, levelups_ref.drop(columns=["player_slug","character_slug"])], axis=1)

* Export

In [13]:
levelups_ref

,player_slug,character_slug,total_purchase,total_ad,highest_level,status,level_from,level_to,event_datetime,death,ad_revival,ad_item,ad_total,purchase,event_datetime_prev,interval,interval_cum,playtime_boss,playtime_dungeon,playtime_tower,playtime_pvp,playtime_farm,playtime_survival,playtime_side
0,2021-10-31T16:01:57.631Z,킹덤,0.0,0.0,7,complete,1,2,2021-11-04 23:42:28,0.0,0.0,0.0,0.0,0.0,2021-11-04 23:41:55,0.009444,0.009444,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-10-31T16:01:57.631Z,킹덤,0.0,0.0,7,complete,2,3,2021-11-13 07:50:45,0.0,0.0,0.0,0.0,0.0,2021-11-04 23:42:28,200.138333,200.147778,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-10-31T16:01:57.631Z,킹덤,0.0,0.0,7,complete,3,4,2021-11-13 07:52:19,0.0,0.0,0.0,0.0,0.0,2021-11-13 07:50:45,0.026389,200.174167,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-10-31T16:01:57.631Z,킹덤,0.0,0.0,7,complete,4,5,2021-11-13 13:11:55,0.0,0.0,0.0,0.0,0.0,2021-11-13 07:52:19,5.326944,205.501111,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-10-31T16:01:57.631Z,킹덤,0.0,0.0,7,complete,5,6,2021-11-13 13:15:02,0.0,0.0,0.0,0.0,0.0,2021-11-13 13:11:55,0.052222,205.553333,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466992,2022-03-30T14:36:22.522Z,킹키킹,0.0,0.0,8,complete,3,4,2022-03-30 23:44:15,0.0,0.0,0.0,0.0,0.0,2022-03-30 23:42:14,0.033889,0.084444,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2466993,2022-03-30T14:36:22.522Z,킹키킹,0.0,0.0,8,complete,4,5,2022-03-30 23:46:19,0.0,0.0,0.0,0.0,0.0,2022-03-30 23:44:15,0.034722,0.119167,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2466994,2022-03-30T14:36:22.522Z,킹키킹,0.0,0.0,8,complete,5,6,2022-03-30 23:47:29,0.0,0.0,0.0,0.0,0.0,2022-03-30 23:46:19,0.019722,0.138889,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2466995,2022-03-30T14:36:22.522Z,킹키킹,0.0,0.0,8,complete,6,7,2022-03-30 23:48:48,0.0,0.0,0.0,0.0,0.0,2022-03-30 23:47:29,0.022222,0.161111,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
levelups_ref.to_csv("../data/levelups_panel_4.csv", index=None, encoding="utf-8")